# Día 4: Abstract Base Classes (ABC)## Descripción GeneralLas **Abstract Base Classes (ABC)** en Python permiten definir interfaces y contratos que las subclases deben cumplir. Son fundamentales para crear arquitecturas extensibles y garantizar que las clases derivadas implementen métodos específicos.El módulo `abc` de Python proporciona la infraestructura para definir clases abstractas, métodos abstractos, y propiedades abstractas.**Conexión con Data/IA**: En proyectos de ML, necesitas múltiples implementaciones de modelos, procesadores de datos, o evaluadores. ABC garantiza que todas las implementaciones cumplan el mismo contrato, permitiendo intercambiarlas sin romper el código.

## Objetivos de AprendizajeAl finalizar este notebook, serás capaz de:1. Crear clases base abstractas usando el módulo abc2. Definir métodos abstractos que las subclases deben implementar3. Usar ABC para definir interfaces y contratos4. Aplicar ABC en diseños de software extensibles5. Comprender cuándo usar ABC vs protocolos6. Desarrollar intuición para identificar cuándo necesitas ABC

## 1. Abstract Base Classes: Definiendo Contratos### 🎯 Contexto: Por Qué Importa**Problema real en Data/IA**: Tienes 5 modelos ML diferentes (RandomForest, XGBoost, NeuralNet, SVM, KNN). Cada uno con métodos diferentes: `train()`, `entrenar()`, `fit()`, `learn()`. **Caos total** 💥 Cambiar de modelo = **reescribir todo el código**. Testing = **mockear cada modelo diferente**. Nuevo desarrollador = **no sabe qué métodos usar**.Sin ABC: Cada modelo hace lo que quiere. `RandomForestModel.train()`, `XGBoostModel.fit()`, `NeuralNetModel.entrenar()`. **Imposible intercambiar modelos** 💥Con ABC: Todos heredan de `BaseModel` con métodos abstractos `train()`, `predict()`, `evaluate()`. **Interfaz común** ✅ Cambiar modelo = **cambiar una línea** ✅ Testing = **mockear una interfaz** ✅**Ejemplo concreto para juniors**:Sistema de pagos con 3 proveedores (Stripe, PayPal, MercadoPago). Sin ABC: `stripe.process_payment()`, `paypal.make_payment()`, `mercadopago.pagar()`. **Cada uno diferente** 💥 Cambiar proveedor = **reescribir lógica de negocio**.Con ABC: Todos implementan `PaymentProcessor` con `process_payment()`, `refund()`, `get_status()`. **Interfaz única** ✅ Cambiar proveedor = **inyectar dependencia diferente** ✅**Consecuencias de NO usarlo**:- **Acoplamiento alto** → código depende de implementaciones específicas ($50K en refactoring)- **Testing imposible** → mockear cada implementación diferente (3x tiempo de testing)- **Bugs multiplicados** → olvidar implementar método crítico (producción se cae)- **Onboarding lento** → nuevos devs no saben qué métodos usar (+2 semanas)- **Refactoring caro** → cambiar interfaz = modificar 20 archivos ($100K)

### 📚 El Concepto**Definición técnica**:**Abstract Base Class (ABC)**: Clase que **no puede instanciarse** y define **métodos abstractos** que las subclases **deben implementar**. Actúa como **contrato** o **interfaz** que garantiza que todas las implementaciones tengan los mismos métodos.**Cómo funciona internamente**:1. Defines clase con `ABC` como metaclass o heredando de `ABC`2. Marcas métodos con `@abstractmethod` decorator3. Python **impide instanciar** la clase abstracta directamente4. Subclases **deben implementar** todos los métodos abstractos5. Si falta un método → **TypeError en tiempo de instanciación**6. Si todos implementados → instanciación exitosa**Terminología clave**:- **Abstract Base Class (ABC)**: Clase que no puede instanciarse, solo heredarse- **Abstract Method**: Método declarado pero no implementado, subclases deben implementarlo- **Concrete Class**: Clase que implementa todos los métodos abstractos (puede instanciarse)- **Contract/Interface**: Conjunto de métodos que una clase promete implementar- **Metaclass**: Clase que define el comportamiento de otras clases (ABC usa metaclass)

### ❌ Ejemplo Incorrecto: Sin ABC**Código**:

In [ ]:
# ❌ BAD: No ABC - inconsistent interfacesfrom typing import List, Dictimport numpy as npclass RandomForestModel:    """Random Forest model - NO ABC."""        def train(self, X: np.ndarray, y: np.ndarray) -> None:        """Train the model."""        print("Training RandomForest...")        self.model = "trained_rf"        def predict(self, X: np.ndarray) -> np.ndarray:        """Make predictions."""        return np.array([1, 0, 1])class XGBoostModel:    """XGBoost model - NO ABC, different interface!"""        def fit(self, X: np.ndarray, y: np.ndarray) -> None:  # Different name!        """Fit the model."""        print("Fitting XGBoost...")        self.model = "trained_xgb"        def predict(self, X: np.ndarray) -> np.ndarray:        """Make predictions."""        return np.array([0, 1, 0])        # Missing evaluate method!class NeuralNetModel:    """Neural Network - NO ABC, completely different!"""        def entrenar(self, X: np.ndarray, y: np.ndarray) -> None:  # Spanish!        """Train the model."""        print("Entrenando Neural Net...")        self.model = "trained_nn"        def predecir(self, X: np.ndarray) -> np.ndarray:  # Spanish!        """Make predictions."""        return np.array([1, 1, 0])# Usage - tightly coupled, can't swap models easilydef train_and_evaluate(model_type: str, X: np.ndarray, y: np.ndarray):    """Train and evaluate - TIGHTLY COUPLED!"""    if model_type == "rf":        model = RandomForestModel()        model.train(X, y)  # train        predictions = model.predict(X)    elif model_type == "xgb":        model = XGBoostModel()        model.fit(X, y)  # fit (different!)        predictions = model.predict(X)    elif model_type == "nn":        model = NeuralNetModel()        model.entrenar(X, y)  # entrenar (Spanish!)        predictions = model.predecir(X)  # predecir (Spanish!)    else:        raise ValueError(f"Unknown model: {model_type}")        return predictions# Testing is a nightmare - need to know each interfaceX = np.array([[1, 2], [3, 4]])y = np.array([0, 1])train_and_evaluate("rf", X, y)

**Problemas**:- **3 interfaces diferentes** → `train()`, `fit()`, `entrenar()`- **Imposible intercambiar** → código acoplado a cada implementación- **Testing pesadilla** → mockear 3 interfaces diferentes- **Bugs silenciosos** → `XGBoostModel` no tiene `evaluate()`, falla en runtime- **Onboarding lento** → nuevos devs deben aprender 3 interfaces- **Refactoring caro** → añadir método = modificar 3 clases manualmente

### ✅ Ejemplo Correcto: Con ABC**Código**:

In [ ]:
# ✅ GOOD: With ABC - consistent interfacefrom abc import ABC, abstractmethodfrom typing import List, Dictimport numpy as npclass BaseModel(ABC):    """    Abstract base class for ML models.    Defines the contract that all models must follow.    """        @abstractmethod    def train(self, X: np.ndarray, y: np.ndarray) -> None:        """        Train the model.                :param X: Training features        :type X: np.ndarray        :param y: Training labels        :type y: np.ndarray        """        pass        @abstractmethod    def predict(self, X: np.ndarray) -> np.ndarray:        """        Make predictions.                :param X: Features to predict        :type X: np.ndarray        :return: Predictions        :rtype: np.ndarray        """        pass        @abstractmethod    def evaluate(self, X: np.ndarray, y: np.ndarray) -> float:        """        Evaluate model performance.                :param X: Test features        :type X: np.ndarray        :param y: True labels        :type y: np.ndarray        :return: Accuracy score        :rtype: float        """        passclass RandomForestModel(BaseModel):    """Random Forest implementation."""        def train(self, X: np.ndarray, y: np.ndarray) -> None:        """Train the Random Forest model."""        print("Training RandomForest...")        self.model = "trained_rf"        def predict(self, X: np.ndarray) -> np.ndarray:        """Make predictions."""        return np.array([1, 0, 1])        def evaluate(self, X: np.ndarray, y: np.ndarray) -> float:        """Evaluate model."""        predictions = self.predict(X)        accuracy = (predictions == y).mean()        return accuracyclass XGBoostModel(BaseModel):    """XGBoost implementation."""        def train(self, X: np.ndarray, y: np.ndarray) -> None:        """Train the XGBoost model."""        print("Training XGBoost...")        self.model = "trained_xgb"        def predict(self, X: np.ndarray) -> np.ndarray:        """Make predictions."""        return np.array([0, 1, 0])        def evaluate(self, X: np.ndarray, y: np.ndarray) -> float:        """Evaluate model."""        predictions = self.predict(X)        accuracy = (predictions == y).mean()        return accuracy# Usage - loosely coupled, easy to swap modelsdef train_and_evaluate(model: BaseModel, X: np.ndarray, y: np.ndarray) -> float:    """    Train and evaluate any model.        :param model: Model instance (must implement BaseModel)    :type model: BaseModel    :param X: Features    :type X: np.ndarray    :param y: Labels    :type y: np.ndarray    :return: Accuracy score    :rtype: float    """    model.train(X, y)    predictions = model.predict(X)    accuracy = model.evaluate(X, y)    return accuracy# Easy to swap models - just change the class!X = np.array([[1, 2], [3, 4], [5, 6]])y = np.array([0, 1, 0])rf_model = RandomForestModel()xgb_model = XGBoostModel()print(f"RF Accuracy: {train_and_evaluate(rf_model, X, y)}")print(f"XGB Accuracy: {train_and_evaluate(xgb_model, X, y)}")# Try to instantiate ABC directly - will fail!try:    base = BaseModel()except TypeError as e:    print(f"\n❌ Cannot instantiate ABC: {e}")

**Ventajas**:- **Interfaz única** → todos usan `train()`, `predict()`, `evaluate()`- **Intercambiable** → cambiar modelo = cambiar una línea- **Testing simple** → mockear una interfaz `BaseModel`- **Bugs imposibles** → olvidar `evaluate()` = TypeError en instanciación- **Onboarding rápido** → aprender una interfaz, usar todos los modelos- **Refactoring barato** → añadir método abstracto = Python fuerza implementación### 📊 Comparación Lado a Lado| Aspecto | Sin ABC | Con ABC ||---------|---------|---------|| Interfaces | 3 diferentes | 1 única || Intercambiabilidad | Imposible | Trivial || Testing | 3 mocks | 1 mock || Bugs en runtime | Frecuentes | Imposibles || Onboarding | 3 interfaces | 1 interfaz || Refactoring | Manual | Forzado por Python || Costo de cambio | $100K | $5K |

### 💡 Aprendizaje Clave**Puntos críticos a recordar**:1. **ABC = contrato** → garantiza que todas las subclases implementen los mismos métodos2. **No instanciable** → ABC solo se hereda, nunca se instancia directamente3. **TypeError en instanciación** → si falta método abstracto, Python falla temprano4. **Intercambiabilidad** → todas las implementaciones son intercambiables5. **Testing simple** → mockear una interfaz, no múltiples implementaciones**Cómo desarrollar intuición**:- **Pregúntate**: "¿Tengo múltiples implementaciones de la misma cosa?"  - SÍ → usa ABC ✅  - NO → no necesitas ABC (todavía)- **Pregúntate**: "¿Necesito intercambiar implementaciones fácilmente?"  - SÍ → usa ABC ✅  - NO → herencia simple está bien- **Pregúntate**: "¿Quiero garantizar que todas las implementaciones tengan los mismos métodos?"  - SÍ → usa ABC ✅  - NO → duck typing está bien- **Pregúntate**: "¿El testing requiere mockear múltiples interfaces?"  - SÍ → usa ABC para unificar ✅  - NO → no necesitas ABC**Cuándo usar / NO usar**:- ✅ **Usar ABC cuando**:  - Múltiples implementaciones de la misma abstracción (modelos ML, procesadores, APIs)  - Necesitas intercambiar implementaciones (Strategy pattern)  - Quieres garantizar interfaz consistente  - Testing requiere mocks de interfaces  - Código de terceros debe implementar tu interfaz- ❌ **NO usar ABC cuando**:  - Solo tienes una implementación (YAGNI - You Aren't Gonna Need It)  - Duck typing es suficiente (Python es dinámico)  - Sobre-ingeniería para casos simples  - Protocolos (typing.Protocol) son más apropiados**Patrones comunes con ABC**:```python# Strategy Patternclass PaymentProcessor(ABC):    @abstractmethod    def process_payment(amount: float) -> bool# Template Method Patternclass DataPipeline(ABC):    def run(self):  # Template method        self.extract()        self.transform()        self.load()        @abstractmethod    def extract(self): pass        @abstractmethod    def transform(self): pass        @abstractmethod    def load(self): pass```**Beneficios medibles**:- **Bugs -80%** → fallos en tiempo de instanciación, no en producción- **Testing +60% más rápido** → mockear una interfaz vs múltiples- **Refactoring +70% más seguro** → Python fuerza implementación- **Onboarding -50% tiempo** → aprender una interfaz vs múltiples**Referencia oficial:** [PEP 3119 - Abstract Base Classes](https://peps.python.org/pep-3119/)

## 2. Métodos Abstractos y Propiedades### 🎯 Contexto: Por Qué Importa**Problema real en Data/IA**: Tienes `DataProcessor` base con 10 métodos. Desarrollador crea `CSVProcessor` pero **olvida implementar** `validate()`. Código compila ✅ Tests pasan ✅ **Producción explota** 💥 Cliente pierde $50K en datos corruptos.Sin `@abstractmethod`: Python no sabe qué métodos son obligatorios. **Bugs silenciosos** 💥 Descubres en runtime (producción).Con `@abstractmethod`: Intentar instanciar `CSVProcessor` sin `validate()` = **TypeError inmediato** ✅ Descubres en desarrollo ✅**Ejemplo concreto para juniors**:Sistema de autenticación con `AuthProvider` base. Métodos: `login()`, `logout()`, `refresh_token()`. Junior crea `GoogleAuthProvider` pero **olvida** `refresh_token()`. **Sesiones expiran** 💥 Usuarios no pueden trabajar 💥Con `@abstractmethod`: Instanciar `GoogleAuthProvider` sin `refresh_token()` = **TypeError** ✅ Junior lo ve inmediatamente ✅**Consecuencias de NO usarlo**:- **Bugs en producción** → métodos faltantes descubiertos tarde ($50K pérdidas)- **Testing incompleto** → tests no cubren métodos faltantes- **Debugging caro** → encontrar por qué falla toma horas ($500/hora)- **Confianza baja** → no sabes si implementación está completa- **Documentación implícita** → no está claro qué métodos son obligatorios

### 📚 El Concepto**Definición técnica**:**Abstract Method**: Método marcado con `@abstractmethod` que **debe ser implementado** por todas las subclases concretas. Python **impide instanciar** clases que no implementan todos los métodos abstractos.**Cómo funciona internamente**:1. Decoras método con `@abstractmethod` en ABC2. Python registra el método como "obligatorio"3. Subclase hereda de ABC4. Al intentar instanciar, Python verifica métodos abstractos5. Si falta alguno → **TypeError con lista de métodos faltantes**6. Si todos implementados → instanciación exitosa**Terminología clave**:- **@abstractmethod**: Decorator que marca método como obligatorio- **@abstractproperty**: Property que debe ser implementada (deprecated, usa @property + @abstractmethod)- **@abstractclassmethod**: Class method abstracto- **@abstractstaticmethod**: Static method abstracto- **Concrete implementation**: Implementación real del método abstracto**Tipos de métodos abstractos**:```pythonfrom abc import ABC, abstractmethodclass Example(ABC):    @abstractmethod    def regular_method(self): pass  # Instance method        @classmethod    @abstractmethod    def class_method(cls): pass  # Class method        @staticmethod    @abstractmethod    def static_method(): pass  # Static method        @property    @abstractmethod    def abstract_property(self): pass  # Property```

### ❌ Ejemplo Incorrecto: Sin @abstractmethod**Código**:

In [ ]:
# ❌ BAD: No @abstractmethod - missing methods not caughtfrom typing import Dict, Listclass DataProcessor:    """Base data processor - NO @abstractmethod!"""        def process(self, data: List[Dict]) -> List[Dict]:        """Process data - should be overridden but not enforced!"""        raise NotImplementedError("Subclasses must implement process()")        def validate(self, data: List[Dict]) -> bool:        """Validate data - should be overridden but not enforced!"""        raise NotImplementedError("Subclasses must implement validate()")        def save(self, data: List[Dict], path: str) -> None:        """Save data - should be overridden but not enforced!"""        raise NotImplementedError("Subclasses must implement save()")class CSVProcessor(DataProcessor):    """CSV processor - FORGETS to implement validate()!"""        def process(self, data: List[Dict]) -> List[Dict]:        """Process CSV data."""        print("Processing CSV...")        return data        def save(self, data: List[Dict], path: str) -> None:        """Save CSV data."""        print(f"Saving to {path}...")        # FORGOT validate() - but Python doesn't complain!# Usage - SILENT BUG!processor = CSVProcessor()  # ✅ Instantiation succeeds!data = [{"id": 1, "value": "test"}]processor.process(data)  # ✅ Worksprocessor.save(data, "output.csv")  # ✅ Works# But calling validate() will crash!try:    processor.validate(data)  # 💥 NotImplementedError in RUNTIME!except NotImplementedError as e:    print(f"\n💥 Runtime error: {e}")    print("This should have been caught at instantiation time!")

**Problemas**:- **Instanciación exitosa** → Python no detecta métodos faltantes- **Bug silencioso** → descubres en runtime, no en desarrollo- **NotImplementedError** → error genérico, no específico- **Testing incompleto** → tests pueden no llamar `validate()`- **Documentación implícita** → no está claro qué métodos son obligatorios- **Debugging caro** → encontrar por qué falla toma tiempo

### ✅ Ejemplo Correcto: Con @abstractmethod**Código**:

In [ ]:
# ✅ GOOD: With @abstractmethod - missing methods caught immediatelyfrom abc import ABC, abstractmethodfrom typing import Dict, Listclass DataProcessor(ABC):    """    Abstract base class for data processors.    All subclasses MUST implement process(), validate(), and save().    """        @abstractmethod    def process(self, data: List[Dict]) -> List[Dict]:        """        Process data.                :param data: Input data        :type data: List[Dict]        :return: Processed data        :rtype: List[Dict]        """        pass        @abstractmethod    def validate(self, data: List[Dict]) -> bool:        """        Validate data.                :param data: Data to validate        :type data: List[Dict]        :return: True if valid        :rtype: bool        """        pass        @abstractmethod    def save(self, data: List[Dict], path: str) -> None:        """        Save data to file.                :param data: Data to save        :type data: List[Dict]        :param path: Output path        :type path: str        """        passclass CSVProcessor(DataProcessor):    """CSV processor - MUST implement all abstract methods."""        def process(self, data: List[Dict]) -> List[Dict]:        """Process CSV data."""        print("Processing CSV...")        return data        def validate(self, data: List[Dict]) -> bool:        """Validate CSV data."""        print("Validating CSV...")        return len(data) > 0        def save(self, data: List[Dict], path: str) -> None:        """Save CSV data."""        print(f"Saving to {path}...")class IncompleteProcessor(DataProcessor):    """Incomplete processor - FORGETS validate()!"""        def process(self, data: List[Dict]) -> List[Dict]:        return data        def save(self, data: List[Dict], path: str) -> None:        print(f"Saving to {path}...")        # FORGOT validate() - Python will catch this!# Usage - complete implementation worksprocessor = CSVProcessor()  # ✅ All methods implementeddata = [{"id": 1, "value": "test"}]processor.process(data)processor.validate(data)processor.save(data, "output.csv")# Incomplete implementation fails at instantiationtry:    incomplete = IncompleteProcessor()  # 💥 TypeError immediately!except TypeError as e:    print(f"\n✅ Caught at instantiation: {e}")    print("Python tells us exactly what's missing!")

**Ventajas**:- **Fallo temprano** → TypeError en instanciación, no en runtime- **Mensaje claro** → Python lista métodos faltantes exactos- **Documentación explícita** → `@abstractmethod` indica obligatoriedad- **Testing completo** → imposible olvidar implementar métodos- **Debugging rápido** → error específico, no genérico- **Confianza alta** → si instancia, está completo### 📊 Comparación Lado a Lado| Aspecto | Sin @abstractmethod | Con @abstractmethod ||---------|---------------------|---------------------|| Detección de errores | Runtime | Instantiation || Mensaje de error | Genérico | Específico || Documentación | Implícita | Explícita || Testing | Puede ser incompleto | Forzado completo || Debugging | Lento | Rápido || Confianza | Baja | Alta || Costo de bug | $50K | $0 |

### 💡 Aprendizaje Clave**Puntos críticos a recordar**:1. **@abstractmethod = obligatorio** → subclases deben implementar2. **Fallo temprano** → TypeError en instanciación, no en runtime3. **Mensaje específico** → Python lista métodos faltantes4. **Documentación explícita** → indica qué métodos son obligatorios5. **Combinar decorators** → `@classmethod` + `@abstractmethod`, `@property` + `@abstractmethod`**Cómo desarrollar intuición**:- **Pregúntate**: "¿Este método DEBE ser implementado por todas las subclases?"  - SÍ → usa `@abstractmethod` ✅  - NO → método concreto con implementación default- **Pregúntate**: "¿Quiero descubrir métodos faltantes en desarrollo o producción?"  - Desarrollo → usa `@abstractmethod` ✅  - Producción (💥) → no uses `@abstractmethod`- **Pregúntate**: "¿Este método tiene implementación default razonable?"  - NO → `@abstractmethod` ✅  - SÍ → método concreto (puede ser overridden)**Cuándo usar / NO usar**:- ✅ **Usar @abstractmethod cuando**:  - Método debe ser implementado por todas las subclases  - No hay implementación default razonable  - Quieres garantizar completitud  - Documentar contrato explícitamente- ❌ **NO usar @abstractmethod cuando**:  - Método tiene implementación default válida  - Método es opcional (no todas las subclases lo necesitan)  - Sobre-restricción innecesaria**Orden de decorators**:```python# ✅ CORRECT order@classmethod@abstractmethoddef method(cls): pass@property@abstractmethoddef prop(self): pass@staticmethod@abstractmethoddef static(): pass# ❌ WRONG order@abstractmethod@classmethod  # Won't work!def method(cls): pass```**Referencia oficial:** [abc Module Documentation](https://docs.python.org/3/library/abc.html)

## 🏋️ Ejercicio: Sistema de Almacenamiento de Modelos**Objetivo**: Practicar creación de ABC con métodos abstractos para un sistema de almacenamiento de modelos ML.**Contexto real**: Estás construyendo un sistema de MLOps que debe guardar modelos en diferentes backends (S3, GCS, Azure Blob, filesystem local). Necesitas una interfaz común para que el código de entrenamiento no dependa del backend específico.**Instrucciones**:1. Crea una ABC llamada `ModelStorage` con métodos abstractos:   - `save_model(model, model_id: str) -> str` (retorna URL)   - `load_model(model_id: str) -> Any` (retorna modelo)   - `delete_model(model_id: str) -> bool` (retorna éxito)   - `list_models() -> List[str]` (retorna IDs de modelos)2. Implementa dos clases concretas:   - `S3ModelStorage`: Simula guardar en S3 (solo prints)   - `LocalModelStorage`: Simula guardar en filesystem (solo prints)3. Crea función `train_and_save(storage: ModelStorage, model, model_id: str)` que:   - Guarda el modelo usando el storage   - Lista todos los modelos   - Retorna la URL**Criterios de éxito**:- [ ] `ModelStorage` es ABC y no puede instanciarse- [ ] Todos los métodos están marcados con `@abstractmethod`- [ ] `S3ModelStorage` y `LocalModelStorage` implementan todos los métodos- [ ] Docstrings en formato Sphinx con type hints- [ ] `train_and_save()` funciona con ambos storages- [ ] Intentar instanciar clase incompleta falla con TypeError

In [ ]:
# TODO: Implement the exercisefrom abc import ABC, abstractmethodfrom typing import Any, List# Your code hereclass ModelStorage(ABC):    """    Abstract base class for model storage backends.        TODO: Add abstract methods    """    pass# TODO: Implement S3ModelStorage# TODO: Implement LocalModelStorage# TODO: Implement train_and_save function# Tests# model = "trained_model_object"# s3_storage = S3ModelStorage(bucket="my-models")# local_storage = LocalModelStorage(base_path="/models")# print("Testing S3 Storage:")# train_and_save(s3_storage, model, "model_v1")# print("\nTesting Local Storage:")# train_and_save(local_storage, model, "model_v1")print("✅ Implement the exercise above!")

<details><summary><b>💡 Pista 1</b></summary>Para crear una ABC:```pythonfrom abc import ABC, abstractmethodclass ModelStorage(ABC):    @abstractmethod    def save_model(self, model: Any, model_id: str) -> str:        pass```Recuerda que todos los métodos deben tener `@abstractmethod`.</details><details><summary><b>💡 Pista 2</b></summary>Para implementar una clase concreta:```pythonclass S3ModelStorage(ModelStorage):    def __init__(self, bucket: str):        self.bucket = bucket        def save_model(self, model: Any, model_id: str) -> str:        # Implementación aquí        return f"s3://{self.bucket}/{model_id}"```Debes implementar TODOS los métodos abstractos.</details><details><summary><b>✅ Ver Solución Completa</b></summary>```pythonfrom abc import ABC, abstractmethodfrom typing import Any, Listclass ModelStorage(ABC):    """    Abstract base class for model storage backends.    Defines the contract for storing, loading, and managing ML models.    """        @abstractmethod    def save_model(self, model: Any, model_id: str) -> str:        """        Save a model to storage.                :param model: Model object to save        :type model: Any        :param model_id: Unique identifier for the model        :type model_id: str        :return: URL or path where model was saved        :rtype: str        """        pass        @abstractmethod    def load_model(self, model_id: str) -> Any:        """        Load a model from storage.                :param model_id: Unique identifier for the model        :type model_id: str        :return: Loaded model object        :rtype: Any        """        pass        @abstractmethod    def delete_model(self, model_id: str) -> bool:        """        Delete a model from storage.                :param model_id: Unique identifier for the model        :type model_id: str        :return: True if deletion was successful        :rtype: bool        """        pass        @abstractmethod    def list_models(self) -> List[str]:        """        List all model IDs in storage.                :return: List of model IDs        :rtype: List[str]        """        passclass S3ModelStorage(ModelStorage):    """S3-based model storage implementation."""        def __init__(self, bucket: str) -> None:        """        Initialize S3 storage.                :param bucket: S3 bucket name        :type bucket: str        """        self.bucket = bucket        self.models: List[str] = []        def save_model(self, model: Any, model_id: str) -> str:        """Save model to S3."""        url = f"s3://{self.bucket}/{model_id}"        print(f"Saving model to {url}")        self.models.append(model_id)        return url        def load_model(self, model_id: str) -> Any:        """Load model from S3."""        print(f"Loading model {model_id} from S3")        return f"model_from_s3_{model_id}"        def delete_model(self, model_id: str) -> bool:        """Delete model from S3."""        print(f"Deleting model {model_id} from S3")        if model_id in self.models:            self.models.remove(model_id)            return True        return False        def list_models(self) -> List[str]:        """List all models in S3."""        return self.modelsclass LocalModelStorage(ModelStorage):    """Local filesystem-based model storage implementation."""        def __init__(self, base_path: str) -> None:        """        Initialize local storage.                :param base_path: Base directory path        :type base_path: str        """        self.base_path = base_path        self.models: List[str] = []        def save_model(self, model: Any, model_id: str) -> str:        """Save model to local filesystem."""        path = f"{self.base_path}/{model_id}.pkl"        print(f"Saving model to {path}")        self.models.append(model_id)        return path        def load_model(self, model_id: str) -> Any:        """Load model from local filesystem."""        print(f"Loading model {model_id} from {self.base_path}")        return f"model_from_local_{model_id}"        def delete_model(self, model_id: str) -> bool:        """Delete model from local filesystem."""        print(f"Deleting model {model_id} from {self.base_path}")        if model_id in self.models:            self.models.remove(model_id)            return True        return False        def list_models(self) -> List[str]:        """List all models in local storage."""        return self.modelsdef train_and_save(storage: ModelStorage, model: Any, model_id: str) -> str:    """    Train and save a model using any storage backend.        :param storage: Storage backend (must implement ModelStorage)    :type storage: ModelStorage    :param model: Model object to save    :type model: Any    :param model_id: Unique identifier for the model    :type model_id: str    :return: URL where model was saved    :rtype: str    """    # Save the model    url = storage.save_model(model, model_id)        # List all models    all_models = storage.list_models()    print(f"All models: {all_models}")        return url# Usagemodel = "trained_model_object"print("Testing S3 Storage:")s3_storage = S3ModelStorage(bucket="my-models")url = train_and_save(s3_storage, model, "model_v1")print(f"Model saved at: {url}\n")print("Testing Local Storage:")local_storage = LocalModelStorage(base_path="/models")path = train_and_save(local_storage, model, "model_v1")print(f"Model saved at: {path}\n")# Try to instantiate ABC directlytry:    base_storage = ModelStorage()except TypeError as e:    print(f"✅ Cannot instantiate ABC: {e}")```**Explicación paso a paso**:1. **ABC con @abstractmethod** → define contrato que todos deben cumplir2. **Clases concretas** → implementan todos los métodos abstractos3. **Función genérica** → acepta cualquier `ModelStorage`, no implementación específica4. **Intercambiabilidad** → cambiar de S3 a Local = cambiar una línea5. **Type safety** → type hint `ModelStorage` garantiza interfaz correcta**Conexión con conceptos**:- **ABC** → define interfaz común para múltiples backends- **@abstractmethod** → garantiza que todos implementen los mismos métodos- **Polimorfismo** → `train_and_save()` funciona con cualquier storage- **Dependency Injection** → inyectar storage permite testing fácil</details>

## Resumen1. **ABC = contrato** → garantiza interfaz consistente entre múltiples implementaciones2. **@abstractmethod** → marca métodos obligatorios, falla temprano si faltan3. **No instanciable** → ABC solo se hereda, TypeError si intentas instanciar4. **Intercambiabilidad** → todas las implementaciones son intercambiables (polimorfismo)5. **Testing simple** → mockear una interfaz ABC, no múltiples implementaciones6. **Usar cuando** → múltiples implementaciones, necesitas garantizar interfaz, Strategy pattern

## Preguntas de Autoevaluación### 1. ¿Cuál es la diferencia entre una clase normal y una ABC?**Respuesta:** Una ABC no puede instanciarse directamente (solo heredarse) y puede tener métodos abstractos que las subclases deben implementar. Python lanza TypeError si intentas instanciar una ABC o una subclase que no implementa todos los métodos abstractos.### 2. ¿Qué pasa si intentas instanciar una clase que hereda de ABC pero no implementa todos los métodos abstractos?**Respuesta:** Python lanza TypeError en el momento de instanciación, listando exactamente qué métodos abstractos faltan. Esto permite detectar errores temprano (desarrollo) en lugar de tarde (producción).### 3. ¿Cuándo deberías usar ABC en lugar de herencia simple?**Respuesta:** Usa ABC cuando tienes múltiples implementaciones de la misma abstracción y necesitas garantizar que todas tengan la misma interfaz. Por ejemplo: múltiples modelos ML, múltiples procesadores de datos, múltiples backends de storage.### 4. ¿Cuál es el orden correcto de decorators para un método abstracto de clase?**Respuesta:** `@classmethod` primero, luego `@abstractmethod`. Ejemplo:```python@classmethod@abstractmethoddef method(cls): pass```### 5. ¿Cuándo NO deberías usar ABC?**Respuesta:** No uses ABC cuando solo tienes una implementación (YAGNI), cuando duck typing es suficiente, o cuando sobre-ingenierías casos simples. También considera `typing.Protocol` para structural subtyping en lugar de nominal subtyping.

## Recursos y Referencias Oficiales### Documentación Oficial- **[abc Module Documentation](https://docs.python.org/3/library/abc.html)**: Documentación oficial del módulo abc  - Contiene todos los decorators y metaclasses disponibles  - Ejemplos de uso y patrones comunes### Estándares/PEPs- **[PEP 3119 - Abstract Base Classes](https://peps.python.org/pep-3119/)**: PEP que introduce ABC en Python  - Motivación y diseño de ABC  - Diferencias con otros lenguajes### Mejores Prácticas- **[Real Python - Abstract Base Classes](https://realpython.com/python-interface/)**: Guía práctica de ABC  - Cuándo usar ABC vs Protocols  - Patrones de diseño con ABC### Herramientas Relacionadas- **[typing.Protocol](https://docs.python.org/3/library/typing.html#typing.Protocol)**: Alternativa a ABC para structural subtyping  - Más flexible que ABC (duck typing)  - No requiere herencia explícita### Notas Importantes- Todos los enlaces están actualizados a partir de 2025- Se recomienda revisar la documentación oficial regularmente- ABC es parte de la standard library desde Python 2.6